#Ehsan_Espandar - 99442011 - Pentago(MINIMAX) - AI


---

My Implementation Following Functions:


*   Rotate Subgrid : which rotates a 3*3 subgrid of the table in + or - directions.
*   Check  Win : which checks if any player has win or not.
*   Evaluate Board : determines which player wins.
*   Generate Possible Moves : Will generate all possible moves in the present state of player.
*   Apply Move and Revert Move : for Applying the chosen move on the board state and revert it.
*   Minimax : which implies the minimax algorithm and makes the minimax tree recursionally
*   best_move: which finds the best path of the minimax tree
*   Parse Human move: It converts the human input into a valid move to be applied on the board.
*   Determine Depth : It determines the depth of minimax tree dynamically as more places on the board are filled.
*   Play Game: It is the main loop of the game.
---



> First I tried to randomize the chosen moves on every depth to be able to discover more depths, but computer at that situation wouldn't be able to see the near winnig actions sometimes.

> Then I found out that I need to discover all moves in a determined depth for a computer to always be able to find winning moves.

>But at the very first when I tried to check depth 2 it costs about 50 milion actions which takes about 8 hours in python So I forgot about it.

>Then I tried to work with depth one until 20 places are empty then try depth 2 which again was 2 million actions and it takes about 1 hours in python.

> So my final result was to try the first move of computer totally random then try depth 1 until more than 14 places are empty then try depth 2 until more than 8 places are empty and then try depth 3 when more than 4 places are empty and finally in other cases try depth 4 which is for the end case.


---

It is needed to be mentioned that computer most of the times wins even before I can reach to the less than 14 places empty, So the program is fine.





In [43]:
import numpy as np
import random
from tqdm import tqdm

# Constants
EMPTY = 0
WHITE = 1
BLACK = 2
SIZE = 6

# Function to rotate a 3x3 subgrid
def rotate_subgrid(board, subgrid_num, direction):
    top_left = {
        1: (0, 0), 2: (0, 3),
        3: (3, 0), 4: (3, 3)
    }[subgrid_num]
    subgrid = board[top_left[0]:top_left[0]+3, top_left[1]:top_left[1]+3]
    if direction == '+':
        subgrid = np.rot90(subgrid, -1)
    elif direction == '-':
        subgrid = np.rot90(subgrid, 1)
    board[top_left[0]:top_left[0]+3, top_left[1]:top_left[1]+3] = subgrid

# Function to check for a win
def check_win(board, player):
    # Check rows
    for i in range(SIZE):
        for j in range(SIZE - 4):
            if np.all(board[i, j:j+5] == player):
                return True

    # Check columns
    for i in range(SIZE):
        for j in range(SIZE - 4):
            if np.all(board[j:j+5, i] == player):
                return True

    # Check diagonals (top-left to bottom-right)
    for i in range(-SIZE + 5, SIZE - 4):
        diagonal = np.diag(board, k=i)
        for j in range(len(diagonal) - 4):
            if np.all(diagonal[j:j+5] == player):
                return True

    # Check diagonals (top-right to bottom-left)
    flipped_board = np.fliplr(board)
    for i in range(-SIZE + 5, SIZE - 4):
        diagonal = np.diag(flipped_board, k=i)
        for j in range(len(diagonal) - 4):
            if np.all(diagonal[j:j+5] == player):
                return True

    return False

# Evaluation function to estimate board state
def evaluate_board(board):
    if check_win(board, WHITE):
        return -10
    elif check_win(board, BLACK):
        return 10
    else:
        return 0

# Generate all possible moves
def generate_possible_moves(board, player):
    moves = []
    for i in range(SIZE):
        for j in range(SIZE):
            if board[i, j] == EMPTY:
                board[i, j] = player
                for k in range(SIZE):
                    for l in range(SIZE):
                        if board[k, l] == EMPTY and (i != k or j != l):
                            board[k, l] = player
                            for subgrid_num in range(1, 5):
                                for direction in ['+', '-']:
                                    moves.append(((i, j), (k, l), subgrid_num, direction))
                            board[k, l] = EMPTY
                board[i, j] = EMPTY
    return moves

# Apply a move to the board
def apply_move(board, move, player):
    (i, j), (k, l), subgrid_num, direction = move
    board[i, j] = player
    board[k, l] = player
    rotate_subgrid(board, subgrid_num, direction)

# Revert a move on the board
def revert_move(board, move):
    (i, j), (k, l), subgrid_num, direction = move
    opposite_direction = '-' if direction == '+' else '+'
    rotate_subgrid(board, subgrid_num, opposite_direction)
    board[i, j] = EMPTY
    board[k, l] = EMPTY

# MiniMax algorithm with alpha-beta pruning
def minimax(board, depth, alpha, beta, maximizing_player):
    eval_score = evaluate_board(board)
    if eval_score != 0 or depth == 0 or np.all(board != EMPTY):
        return eval_score

    if maximizing_player:
        max_eval = -np.inf
        moves = generate_possible_moves(board, BLACK)
        for move in moves:
            apply_move(board, move, BLACK)
            eval = minimax(board, depth - 1, alpha, beta, False)
            revert_move(board, move)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = np.inf
        moves = generate_possible_moves(board, WHITE)
        for move in moves:
            apply_move(board, move, WHITE)
            eval = minimax(board, depth - 1, alpha, beta, True)
            revert_move(board, move)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

# Function to find the best move using Minimax
def best_move(board, depth, player):
    best_eval = -np.inf if player == BLACK else np.inf
    best_moves = []
    moves = generate_possible_moves(board, player)
    for move in tqdm(moves, desc="Best Move", leave=False):
        apply_move(board, move, player)
        move_eval = minimax(board, depth - 1, -np.inf, np.inf, player == WHITE)
        revert_move(board, move)
        if player == BLACK and move_eval > best_eval:
            best_eval = move_eval
            best_moves = [move]
        elif player == WHITE and move_eval < best_eval:
            best_eval = move_eval
            best_moves = [move]
        elif move_eval == best_eval:
            best_moves.append(move)
    return best_moves

# Function to parse human move
def parse_human_move(move_str):
    move1 = (int(move_str[0]) - 1, int(move_str[1]) - 1)
    move2 = (int(move_str[2]) - 1, int(move_str[3]) - 1)
    subgrid_num = int(move_str[4])
    direction = move_str[5]
    return move1, move2, subgrid_num, direction

# Determine dynamic depth based on game progress
def determine_depth(board):
    empty_cells = np.sum(board == EMPTY)
    if empty_cells > 16:
        return 1
    elif empty_cells > 10:
        return 2
    elif empty_cells > 4:
        return 3
    else:
        return 4

# Main game loop
def play_game():
    board = np.zeros((SIZE, SIZE), dtype=int)
    current_player = BLACK  # Computer starts the game
    first_move = True
    while not check_win(board, WHITE) and not check_win(board, BLACK) and np.any(board == EMPTY):
        print(board)
        if current_player == BLACK:
            if first_move:
                # Random move for the first move
                empty_positions = list(zip(*np.where(board == EMPTY)))
                move1, move2 = random.sample(empty_positions, 2)
                subgrid_num = random.choice([1, 2, 3, 4])
                direction = random.choice(['+', '-'])
                move = (move1, move2, subgrid_num, direction)
                first_move = False
            else:
                depth = determine_depth(board)
                best_moves = best_move(board, depth, BLACK)
                if best_moves:
                    move = random.choice(best_moves)
            apply_move(board, move, BLACK)
            print(f"Computer move: {move[0][0]+1}{move[0][1]+1}{move[1][0]+1}{move[1][1]+1}{move[2]}{move[3]}")
            current_player = WHITE
        else:
            move_str = input("Enter your move (e.g., '23362+'): ")
            move1, move2, subgrid_num, direction = parse_human_move(move_str)
            if board[move1[0], move1[1]] != EMPTY or board[move2[0], move2[1]] != EMPTY:
                print("Invalid move. One of the positions is already occupied.")
                continue
            move = (move1, move2, subgrid_num, direction)
            apply_move(board, move, WHITE)
            current_player = BLACK
    print(board)
    if check_win(board, WHITE):
        print("Human wins!")
    elif check_win(board, BLACK):
        print("Computer wins!")
    else:
        print("It's a draw!")

# Run the game
play_game()


[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
Computer move: 44242+
[[0 0 0 0 2 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 2 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
Enter your move (e.g., '23362+'): 11124-
[[1 1 0 0 2 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 2 0 0]]


Computer move: 25214+
[[1 1 0 0 2 0]
 [2 0 0 0 2 0]
 [0 0 0 0 0 0]
 [0 0 0 2 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
Enter your move (e.g., '23362+'): 22354-
[[1 1 0 0 2 0]
 [2 1 0 0 2 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 2 0 0]]


Computer move: 53514+
[[1 1 0 0 2 0]
 [2 1 0 0 2 0]
 [0 0 0 0 1 0]
 [0 0 0 2 0 0]
 [2 0 2 0 0 0]
 [0 0 0 0 0 0]]
Enter your move (e.g., '23362+'): 52344-
[[1 1 0 0 2 0]
 [2 1 0 0 2 0]
 [0 0 0 1 1 0]
 [0 0 0 0 0 0]
 [2 1 2 0 0 0]
 [0 0 0 2 0 0]]


Computer move: 54634+
[[1 1 0 0 2 0]
 [2 1 0 0 2 0]
 [0 0 0 1 1 0]
 [0 0 0 2 2 0]
 [2 1 2 0 0 0]
 [0 0 2 0 0 0]]
Enter your move (e.g., '23362+'): 43463+
[[1 1 0 0 2 0]
 [2 1 0 0 2 0]
 [0 0 0 1 1 0]
 [0 2 0 2 2 1]
 [0 1 0 0 0 0]
 [2 2 1 0 0 0]]


Computer move: 41432-
[[1 1 0 0 0 0]
 [2 1 0 2 2 1]
 [0 0 0 0 0 1]
 [2 2 2 2 2 1]
 [0 1 0 0 0 0]
 [2 2 1 0 0 0]]
Computer wins!
